In [ ]:
!pip install torch torchvision torchaudio
!pip install roboflow
!pip install matplotlib numpy
!pip install roboflow


  Using cached torch-2.8.0-cp311-cp311-win_amd64.whl.metadata (30 kB)
  Using cached torchvision-0.23.0-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
  Using cached torchaudio-2.8.0-cp311-cp311-win_amd64.whl.metadata (7.2 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached MarkupSafe-3.0.2-cp311-cp311-win_amd64.whl.metadata (4.1 kB)
Using cached torch-2.8.0-cp311-cp311-win_amd64.whl (241.4 MB)
Using cached torchvision-0.23.0-cp311-cp311-win_amd64.whl (1.6 MB)
Using cached torchaudio-2.8.0-cp311-cp311-win_amd64.whl (2.5 MB)
Using cached filelock-3.19.1-py3-none-any.whl (15 kB)
Using cached fsspec-2025.9.0-py3-none-any.whl (199 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
Using cached MarkupSafe-3.0.2-cp311-cp311-win_amd64.whl (15 kB)


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\naval\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\torchvision\\_C.pyd'
Check the permissions.


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\naval\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\naval\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\naval\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="AWN4nVlJhx52ROuzjIPX")
project = rf.workspace("itrash").project("apple-khthj")
version = project.version(4)
dataset = version.download("coco")
                


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\naval\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Apple-4 in coco:: 100%|██████████| 3383/3383 [00:07<00:00, 465.62it/s]
